In [7]:
import pandas as pd
from collections import defaultdict
import os
import numpy as np
import operator
import re

In [4]:
topics = defaultdict(list,
            {1: ['money', 'black'],
             2: ['sends','corruption','back','bring','fight','white','abroad','payer'],
             3: ['gdp', 'growth', 'rate', 'quarter'],
             4: ['high','per','increased','rise','increase','reduce','highest','cent','low'],
             5: ['oil', 'excise', 'rising', 'vat', 'duty'],
             6: ['price', 'hike', 'crude'],
             7: ['narendra', 'finance', 'minister', 'prime', 'chief', 'shri'],
             8: ['muslim', 'empowerment', 'equality'],
             9: ['economy','year','economic','job','inflation','indian','last','deficit','nmapp','via','base','export','fiscal','strong','current','growing','credit','gst','rupee','central','reduced',
                 'cost','interest','subsidy','meeting','tax','one','income','nation','collection','department','market','council','simple','lpg','story','level','cylinder','household'],
             10: ['gujarat','election','result','defeat','sonia','realize','utter','court','supreme','bill','sabha','historic','tripletalaq','tripletalaqbill','rajya','triple','lok','talaq',
                  'respect','justice','passed'],
             11: ['demonetisation','employment','note','currency','ban','demonetised','bank','crore','people','day','campaign','action','taken','system','lakh','promise','come','name','trader','return'],
             12: ['world','sector','effect','good','month','since','global','government','public','yet','state','across','see','true','single','benefit','delhi','waiver','poor','debt','man','common',
                  'protest','due','pay','session','road','direct','relief','revenue','gstfornewindia','welfare','problem','demand','implementation','small','budget','service','class','middle','connection','free','pmuy','ujjwala'],
             13: ['commission','fodder','tamil','ram','ayodhya','temple','mandir','jai'],
             14: ['gandhi', 'sale', 'rahul'],
             15: ['din','business','country','first','big','like','pnbscam','bjp','could','also','report','every','run','data','make','farmer','million','need','company','help','save','conference','twitter'],
             16: ['niravmodi','fraud','loan','working','democracy','swachhatahiseva','app','said','bihar','sir','would','video','work','must','karnataka','film','singh','land'],
             17: ['fdi', 'policy'],
             18: ['investment', 'new', 'property'],
             19: ['even', 'govt', 'time'],
             20: ['long', 'give'],
             21: ['say', 'dont'],
             22: ['card'],
             23: ['acche'],
             24: ['pnb', 'prasad', 'lalu', 'management', 'com'],
             25: ['padmavati','many','beef','today','evm','vote','towards','already','demo','aadhaar','aadhar','case','decision','leader','life','stand','parliament','party','issue','special',
                  'assembly','water','cauvery','board','order','hindu','cauveryissue','congress','development','welcome','woman','right','without','law','three','obc','president','national',
                  'thanks','community','backward','yojana'],
             26: ['india', 'clean', 'movement', 'cleanliness'],
             27: ['youth', 'step'],
             28: ['take', 'get', 'let', 'opposition', 'know', 'scheme'],
             29: ['question', 'demonetization', 'made', 'stop', 'family'],
             30: ['scam'],
             31: ['modi', 'nirav', 'mallya', 'mehul', 'choksi', 'vijay'],
             32: ['pic'],
             33: ['beneficiary'],
             34: ['reform','etc','soon','agent','inclusion','removal','savesonia','bankdepositloot'],
             35: ['item'],
             36: ['success', 'tomorrow', 'next', 'sharing'],
             37: ['competing', 'newgen', 'entrepreneursleaders'],
             38: ['petrol', 'diesel', 'liter']})

In [21]:
dataset = pd.read_csv('../data/Labelled_tweets_v1.csv')

In [22]:
tweets = dataset['Tweet'].values.tolist()

In [23]:
for i in range(len(tweets)):
    tweets[i] = tweets[i].split()

In [8]:
topics_size = {}
for i in range(len(topics)):
    topics_size[i+1] = len(topics[i+1])

In [48]:
topics_assigned = []
match_all = []

In [49]:
for tweet in tweets:
    match = {}
    top_two = []
    for word in set(tweet):
        for i in range(len(topics)):
            if word in set(topics[i]):
                if i in match:
                    match[i] += 1
                else:
                    match[i] = 1
    for i in match.keys():
        precision = match[i]/topics_size[i]
        recall = match[i]/len(set(tweet))
        jc=match[i]/(len(set(tweet))+topics_size[i])
        match[i] = jc
    sorted_match = sorted(match.items(), key=operator.itemgetter(1),reverse = True)
    #print(sorted_match)
    if(len(sorted_match)==0):
        top_two = [0,0]
    elif(len(sorted_match)<2):
        #print(sorted_match)
        top_two.append(sorted_match[0][0])
        top_two.append(0)
    else:
        for j in range(2):
            top_two.append(sorted_match[j][0])
    topics_assigned.append(top_two)
    match_all.append(match)

In [1]:
# alpha = 0.5
# beta = 0.5
# for i in match.keys():
#     print(i)
#     precision = match[i]/topics_size[i]
#     recall = match[i]/len(set(tweet))
#     jc=match[i]/(len(set(tweet))+topics_size[i])
#     #print(np.sqrt(precision*recall))
#     #print(precision,recall)
#     print(jc)
#     print(2*(precision*recall)/(precision+recall))
# #print(sorted(match.items()))
# print(match)
# #print(tweets[0])

In [50]:
topics_assigned[0:5]

[[11, 15], [12, 25], [15, 30], [16, 15], [31, 16]]

In [51]:
print(tweets[3])

['would', 'bjp', 'confirm', 'this', 'if', 'true', 'what', 'transpired', 'in', 'that', 'meeting', 'breaking', 'my', 'sources', 'tell', 'me', 'that', 'niravmodi', 'his', 'brother', 'had', 'a', 'privileged', 'private', 'meeting', 'with', 'USER_MENTION', 'in', 'davos']


In [53]:
topics_assigned = pd.DataFrame(topics_assigned)
topics_assigned.to_csv('topics_assigned_clsuter.csv',index=False, header = False)

In [5]:
lda_topics = open('../twitter-lda/lda_topics.txt').readlines()

In [9]:
for i in range(len(lda_topics)):
    lda_topics[i] = re.sub('\n',' ',lda_topics[i])

In [17]:
all_topics = defaultdict(list)
keywords = []
count = 1
for i in range(len(lda_topics)):
    
    if(re.match('--',lda_topics[i])):
        continue
    elif(lda_topics[i] == ' '):
        all_topics[count] = keywords
        count += 1
        keywords = []
    else:
        keywords.append(lda_topics[i])
    


In [39]:
topics_assigned_lda =[]
match_all_lda =[]

In [48]:
for tweet in tweets:
    match = {}
    top_two = []
    for word in set(tweet):
        for i in range(len(all_topics)):
            if word+' ' in all_topics[i]:
                #print('adf')
                if i in match:
                    match[i] += 1
                else:
                    match[i] = 1
    for i in match.keys():
        precision = match[i]/len(all_topics[i])
        recall = match[i]/len(set(tweet))
        jc=match[i]/(len(set(tweet))+len(all_topics[i]))
        match[i] = jc
    sorted_match = sorted(match.items(), key=operator.itemgetter(1),reverse = True)
    #print(sorted_match)
    if(len(sorted_match)==0):
        top_two = [0,0]
    elif(len(sorted_match)<2):
        #print(sorted_match)
        top_two.append(sorted_match[0][0])
        top_two.append(0)
    else:
        for j in range(2):
            top_two.append(sorted_match[j][0])
    topics_assigned_lda.append(top_two)
    match_all_lda.append(match)

In [46]:
'demonetisation'+' ' in all_topics[1]

True

In [49]:
topics_assigned_lda

[[4, 18],
 [4, 24],
 [4, 8],
 [8, 4],
 [4, 20],
 [4, 21],
 [8, 21],
 [8, 2],
 [26, 2],
 [23, 13],
 [26, 21],
 [19, 23],
 [26, 16],
 [7, 5],
 [12, 26],
 [20, 17],
 [20, 24],
 [20, 28],
 [20, 13],
 [20, 17],
 [15, 20],
 [15, 20],
 [20, 16],
 [20, 16],
 [20, 17],
 [20, 17],
 [20, 17],
 [20, 17],
 [20, 17],
 [20, 17],
 [20, 21],
 [19, 23],
 [20, 12],
 [21, 17],
 [17, 20],
 [28, 19],
 [2, 5],
 [22, 24],
 [23, 8],
 [23, 19],
 [6, 1],
 [3, 18],
 [22, 16],
 [4, 10],
 [7, 4],
 [10, 9],
 [28, 26],
 [18, 14],
 [18, 24],
 [9, 28],
 [21, 16],
 [28, 9],
 [24, 14],
 [24, 23],
 [24, 22],
 [10, 8],
 [24, 14],
 [24, 16],
 [24, 22],
 [24, 0],
 [24, 7],
 [24, 0],
 [7, 18],
 [24, 15],
 [24, 16],
 [24, 7],
 [24, 1],
 [24, 20],
 [20, 24],
 [16, 8],
 [24, 4],
 [24, 7],
 [24, 0],
 [27, 24],
 [24, 25],
 [24, 21],
 [24, 28],
 [24, 1],
 [24, 10],
 [24, 16],
 [9, 28],
 [19, 4],
 [2, 17],
 [7, 10],
 [14, 22],
 [7, 19],
 [1, 19],
 [21, 14],
 [13, 5],
 [18, 25],
 [18, 6],
 [18, 7],
 [6, 1],
 [18, 6],
 [18, 14],
 [19,

In [51]:
topics_assigned_lda = pd.DataFrame(topics_assigned_lda)

In [52]:
topics_assigned_lda.to_csv('topics_assigned_lda.csv',header=False, index = False)

In [53]:
all_topics[4]

['modi ',
 'bank ',
 'crore ',
 'pm ',
 'nirav ',
 'scam ',
 'pnb ',
 'government ',
 'niravmodi ',
 'country ',
 'like ',
 'pnbscam ',
 'r ',
 'fraud ',
 'say ',
 'take ',
 'bjp ',
 'loan ',
 'people ',
 'govt ',
 'mallya ',
 'public ',
 'mehul ',
 'time ',
 'first ',
 'choksi ',
 'vijay ',
 'could ',
 'gandhi ',
 'yet ',
 'narendra ']

In [55]:
print(tweets[0:3])

[['a', 'no', 'of', 'people', 'approach', 'me', 'daily', 'worried', 'abt', 'the', 'safety', 'of', 'their', 'money', 'in', 'banks', 'considering', 'large', 'no', 'of', 'bank', 'frauds', 'coming', 'to', 'light', 'daily', 'pm', 'may', 'kindly', 'inform', 'the', 'public', 'which', 'bank', 'is', 'safe', 'for', 'people', 'to', 'keep', 'their', 'money', 'URL', 'sir', 'there', 'is', 'a', 'big', 'rumour', 'people', 'are', 'very', 'skeptical', 'that', 'their', 'saved', 'money', 'in', 'the', 'banks', 'can', 'go', 'for', 'a', 'toss', 'anytime', 'hope', 'wish', 'pray', 'that', 'you', 'our', 'people', 'all', 'of', 'us', 'save', 'the', 'hard', 'earned', 'money', 'in', 'banks', 'improve', 'our', 'incomes', 'strengthen', 'the', 'income', 'tax', 'system'], ['its', 'now', 'revealed', 'that', 'our', 'fms', 'silence', 'on', 'the', 'pnb', 'scam', 'was', 'to', 'protect', 'his', 'lawyer', 'daughter', 'who', 'was', 'paid', 'a', 'large', 'retainer', 'by', 'the', 'accused', 'just', 'a', 'month', 'before', 'the', 